<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_0618/blob/main/tibame_practice_momo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dirtyjson

In [ ]:
import urllib.request as req
import bs4 as bs
import json
import dirtyjson

url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())

scripts = html.find_all("script")
target_script = None
for script in scripts:
    if "@type" in script.text:
        target_script = script
        break
datas = dirtyjson.loads(target_script.text)
f = open("data.json", "w", encoding="utf-8")
json.dump(datas, f, indent=4, ensure_ascii=False)
f.close()

In [9]:
import urllib.request as req
import bs4 as bs

def get_product_info(url):
    try:
        # url = "https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=7597129&Area=search&mdiv=403&oid=1_7&cid=index&kw=%E5%90%B9%E9%A2%A8%E6%A9%9F"
        resp = req.urlopen(url)
        html = bs.BeautifulSoup(resp.read())

        brand = html.find("a", {"id":"webBrand"})
        info_ul = html.find("ul", {"id":"categoryActivityInfo"})
        if not info_ul == None:
            infos = info_ul.find_all("li")
        else:
            infos = []
        prices = html.find("ul", {"class":"prdPrice"}).find_all("li")
        if len(prices) == 3:
            price_original = prices[0].find("del")
            price_discount = prices[1].find("del")
            price_special = prices[2].find("span")
        elif len(prices) == 2:
            price_original = prices[0].find("del")
            price_discount = prices[1].find("span")
            price_special = None
        else:
            price_original = None
            price_discount = prices[0].find("span")
            price_special = None

        if not brand == None:
            brand_text = brand.text
        else:
            brand_text = "-"
        # price
        if not price_original == None:
            price_original_text = price_original.text
        else:
            price_original_text = "-"
        price_discount_text = price_discount.text
        if not price_special == None:
            price_special_text = price_special.text
        else:
            price_special_text = "-"
        # info
        info_text = ""
        for info in infos:
            info_text = info_text + info.text + "\n"

        # print(brand_text)
        # print(price_original_text)
        # print(price_discount_text)
        # print(price_special_text)
        # print(info_text)

        row = {
            "brand":brand_text,
            "price_original":price_original_text,
            "price_discount":price_discount_text,
            "price_special":price_special_text,
            "goods_info":info_text,
        }
        return row
    except:
        return None

In [10]:
import urllib.request as req
import bs4 as bs
import json
import dirtyjson
import pandas as pd

def get_page_products(url):
    # url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0&curPage=1"
    resp = req.urlopen(url)
    html = bs.BeautifulSoup(resp.read())

    scripts = html.find_all("script")
    target_script = None
    for script in scripts:
        if "@type" in script.text:
            target_script = script
            break
    datas = dirtyjson.loads(target_script.text)

    table = []
    items = datas["mainEntity"]["itemListElement"]
    for item in items:
        item_name = item["name"]
        item_image = item["image"]
        item_url = item["url"]
        item_price = item["offers"]["price"]
        print(item_name)
        print(item_image)
        print(item_url)
        print(item_price)
        # 你要再進去每個item的url
        # 介紹/品牌/三個價格
        row = get_product_info(item_url)
        if not row == None:
            row["name"] = item_name
            row["image"] = item_image
            row["url"] = item_url
            table.append(row)
    return table



total_table = []
for i in range(3):
    page = i + 1
    url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0&curPage=" + str(page)
    partial = get_page_products(url)
    total_table.extend(partial)
df = pd.DataFrame(total_table)
df.to_csv("momo.csv", encoding="utf-8")


【dyson 戴森】HD08 Origin Supersonic 吹風機 溫控 負離子(瑰麗紅色 平裝版)
https://img1.momoshop.com.tw/goodsimg/0012/749/140/12749140_OL.jpg?t=1751612057
https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12749140&Area=search&mdiv=403&oid=1_1&cid=index&kw=%E5%90%B9%E9%A2%A8%E6%A9%9F
7999
【小不記】可折疊高速負離子吹風機 強勁大風量(智能控溫護髮 速乾 附吹風機收納架)
https://img1.momoshop.com.tw/goodsimg/0011/977/069/11977069_OL.jpg?t=1744369723
https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11977069&Area=search&mdiv=403&oid=1_2&cid=index&kw=%E5%90%B9%E9%A2%A8%E6%A9%9F
555
【KINYO】BLDC高速無刷負離子吹風機KH-9688(11萬轉速2億負離子)
https://img1.momoshop.com.tw/goodsimg/0013/880/987/13880987_OL.jpg?t=1749099769
https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13880987&Area=search&mdiv=403&oid=1_3&cid=index&kw=%E5%90%B9%E9%A2%A8%E6%A9%9F
1680
【MIDORI 美多莉】Hypersonic高速負離子吹風機-霧夜金(全配組-不含收納架)
https://img1.momoshop.com.tw/goodsimg/0011/280/148/11280148_OL.jpg?t=1735880731
https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11280148&Area=sear